In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import json

# Dataset path
dataset_dir = r"D:\cattle"

#  Parameters
img_size = (224, 224)
batch_size = 32
validation_split = 0.2  # 20% for validation
seed = 123
AUTOTUNE = tf.data.AUTOTUNE

#  Load training dataset from same directory with split
train_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "Cow"),
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=True
)

#  Load validation dataset from same directory with split
val_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "Cow"),
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="int",
    shuffle=True
)

#  Get number of classes
num_classes = len(train_ds.class_names)
print("Detected classes:", train_ds.class_names)

#  Optional: Save class names for later use
with open("breed_class_names.json", "w") as f:
    json.dump(train_ds.class_names, f)

#  Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

#  Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

#  Load EfficientNetB0 as base model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base model

#  Build model
inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

#  Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#  Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30
)

#  Save model in both formats
model.save("Cow_breed.keras")  # Recommended format
model.save("Cow_breed.h5")     # Legacy format

print(" Model saved successfully.")


Found 972 files belonging to 5 classes.
Using 778 files for training.
Found 972 files belonging to 5 classes.
Using 194 files for validation.
Detected classes: ['Ayshire', 'Brown_swiss', 'Holstein_Friesian', 'Khillari', 'Red_Sindhi']
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.1746 - loss: 1.7645 - val_accuracy: 0.2835 - val_loss: 1.5603
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.2918 - loss: 1.5724 - val_accuracy: 0.4227 - val_loss: 1.4188
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.3486 - loss: 1.4977 - val_accuracy: 0.5258 - val_loss: 1.3053
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.4628 - loss: 1.3595 - val_accuracy: 0.6134 - val_loss: 1.2146
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5065 - loss: 1.2765 - val_accuracy: 0.6443 - val_loss: 1.1367
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5594 - loss: 1.2189 - val_accuracy: 0.7165 - val_loss: 1.0684
Epoch 7/30

✅ Model saved successfully.


In [ ]:
how to use the model

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import os

# Define your class names in the same order used during training
class_names = ['Ayshire', 'Brown_swiss', 'Holstein_Friesian', 'Khillari', 'Red_Sindhi'] # <-- update to match your dataset

def predict_image(path, model):
    img = image.load_img(path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    # img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)    //////  this convertion loses the data and give wrong pridiction
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    print(f"Prediction scores: {predictions}")
    return predicted_class

# Load model
model = tf.keras.models.load_model("Cow_breed.keras")

# Predict
print(predict_image("red sindhi.png", model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction scores: [[0.08349954 0.06474439 0.02190582 0.05707481 0.7727754 ]]
Red_Sindhi


In [ ]:
model.summary()